In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import mne
import matplotlib.pyplot as plt
import numpy as np
import sys
import re
import random
import tensorflow as tf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
def save_the_channels():
    file = "MIT_CHB/chb01/chb01_01.edf"
    data = mne.io.read_raw_edf(file)
    channels = data.ch_names
    channels_list = channels[:14] + channels[15:18]
    channels_list.sort()
    print("Total Channels num :", len(channels_list))
    print("Channels : ", channels_list)
    return channels_list

def plot_by_channels(file, channels_list):
    data = mne.io.read_raw_edf(file)
    data = data.pick_channels(channels_list)
    data = data.reorder_channels(channels_list)
    data.plot(start=2990, duration=10)
    print(data.copy().crop(tmin=0, tmax=23).get_data())

def save_the_seizure_block():
    seizure_file = []
    seizure_time = []
    for num in range(1,25):
        file = "chb"+str(num).rjust(2, '0')

        with open("MIT_CHB/"+ file + "/" + file + "-summary.txt") as txt:
            text = txt.readlines()
            for content in text:
                if file in content:
                    edf_file_name = content.split(" ")[-1].replace("\n", "")
                    seizure_file.append(edf_file_name)

                # count seizures
                if "Number of Seizures" in content:
                    seizure_cnt = int(content[-2])
                    if seizure_cnt == 0:
                        seizure_file.pop()
                    else:
                        for _ in range(seizure_cnt - 1):
                            seizure_file.append(edf_file_name)
                            
                start_re = re.compile('Seizure.+Start Time')
                end_re = re.compile('Seizure.+End Time')
                
                if start_re.match(content):
                    start_time = int(re.findall("\d+", content.split(":")[-1])[0])

                if end_re.match(content):
                    end_time = int(re.findall("\d+", content.split(":")[-1])[0])
                    seizure_time.append((start_time, end_time))
                    # except chb12 - 27, 28, 29
                    if 'chb12_27' in edf_file_name or 'chb12_28' in edf_file_name or 'chb12_29' in edf_file_name:
                        seizure_file.pop()
                        seizure_time.pop()

    seizure_block = []  
    for i in range(len(seizure_file)):
        seizure_block.append((seizure_file[i], seizure_time[i]))

    return seizure_block

def save_the_segmentation(file, channels_list, seizure_block):
    global seizure_seg
    global non_seizure_seg
    global y_label
    file = "MIT_CHB" + "/" + file.replace("\n", "")
    
    if "12_27" in file or "12_28" in file or "12_29" in file:
        return
    
    seizure_check = False
    data = mne.io.read_raw_edf(file)
    pick_chans_data = data.pick_channels(channels_list)
    reorder_chans_data = pick_chans_data.reorder_channels(channels_list)
    
    seizure_time = []
    for block in seizure_block:
        if file.split("/")[-1] == block[0]:
            seizure_check = True
            seizure_time.append(block[1])

    if (seizure_check):
        # except last segment (because short length)
        for i in range(reorder_chans_data.get_data().copy().shape[1]//256//23 - 1):
            for j in seizure_time:
                if ((j[0] <= i*23 and i*23 < j[1]) or (j[0] <= (i+1)*23 and (i*23)+1 < j[1])):
                    print("Segment block time :", i*23, "~", (i+1)*23, ", Seizure Time : ", j[0], "~", j[1])
                    seizure_seg = np.append(seizure_seg, np.reshape(reorder_chans_data.copy().crop(tmin=i*23, tmax=(i+1)*23).get_data(), (1, 17, 5889)), axis=0)
                    y_label = np.append(y_label, 1)
            # Regulate non_seizure_segment_Count
            if np.random.random((1)) > 0.004:
                continue
            non_seizure_seg = np.append(non_seizure_seg, np.reshape(reorder_chans_data.copy().crop(tmin=i*23, tmax=(i+1)*23).get_data(), (1, 17, 5889)), axis=0)
            y_label = np.append(y_label, 0)  
    else:
        for i in range(reorder_chans_data.get_data().copy().shape[1]//256//23 - 1):
            # Regulate non_seizure_segment_Count
            if np.random.random((1)) > 0.004:
                continue
            non_seizure_seg = np.append(non_seizure_seg, np.reshape(reorder_chans_data.copy().crop(tmin=i*23, tmax=(i+1)*23).get_data(), (1, 17, 5889)), axis=0)
            y_label = np.append(y_label, 0)
        
# save_the_segmentation('chb01/chb01_15.edf', save_the_channels(), save_the_seizure_block())

In [3]:
seizure_seg = np.empty((1, 17,5889), dtype=np.float64)
non_seizure_seg = np.empty((1, 17,5889), dtype=np.float64)
y_label = np.array((0), dtype=np.int8)

In [4]:
chans = save_the_channels()
blocks = save_the_seizure_block()
with open("MIT_CHB/RECORDS.txt", 'r') as records_f:
    content = records_f.readlines()
    for i in content[:]:
        save_the_segmentation(i, chans, blocks)

Extracting EDF parameters from C:\Users\user\Desktop\개인공부\뇌공학\MIT_CHB\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Total Channels num : 17
Channels :  ['C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8', 'FP1-F3', 'FP1-F7', 'FP2-F4', 'FP2-F8', 'FZ-CZ', 'P3-O1', 'P4-O2', 'P7-O1', 'P8-O2', 'T7-P7']
Extracting EDF parameters from C:\Users\user\Desktop\개인공부\뇌공학\MIT_CHB\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\user\Desktop\개인공부\뇌공학\MIT_CHB\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\Users\user\Desktop\개인공부\뇌공학\MIT_CHB\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Segment block time : 2990 ~ 3013 , Seizure Time :  2996 ~ 3036
Segment block time : 3013 ~ 3036 , Seizure 

In [5]:
seizure_seg = np.delete(seizure_seg, 0, axis=0)
non_seizure_seg = np.delete(non_seizure_seg, 0, axis=0)
y_label = np.delete(y_label, 0)

In [6]:
print(seizure_seg.shape)
print(non_seizure_seg.shape)
print(y_label.shape)

(662, 17, 5889)
(620, 17, 5889)
(1282,)


In [7]:
seizure_seg = np.append(seizure_seg, non_seizure_seg, axis=0)

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(seizure_seg, y_label, test_size=0.3)
# x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, random_state=66, test_size=0.5)

In [9]:
y_label = tf.keras.utils.to_categorical(y_label, 2)

In [8]:
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

NameError: name 'y_train' is not defined

In [ ]:
x_train.shape, x_test.shape

In [10]:
# import tensorflow as tf
# from tensorflow import keras
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Bidirectional, LSTM, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras import activations

model = Sequential()
model.add(SeqSelfAttention(attention_activation='sigmoid', input_shape=(17, 5889)))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(TimeDistributed(Dense(50)))
model.add(GlobalAveragePooling1D())
model.add(Dense(2, activation='softmax'))
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)
# model.summary()

# history = model.fit(x_train, y_train, epochs=50, batch_size=30)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10)
cv_accuracy = []
n_iter =0
for train_index, test_index in kfold.split(seizure_seg):
    x_train, x_test = seizure_seg[train_index], seizure_seg[test_index]
    y_train, y_test = y_label[train_index], y_label[test_index]
    model.fit(x_train, y_train, epochs=50, batch_size=30)
    pred = model.predict(x_test)
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4) # 소수점 4자리 반올림
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

Epoch 1/50


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/bidirectional/forward_lstm/PartitionedCall]] [Op:__inference_train_function_6518]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Bidirectional, LSTM, TimeDistributed, GlobalAveragePooling1D, Dropout,RepeatVector
from tensorflow.keras import activations
model = Sequential()
model.add(LSTM(128, input_shape=(17, 5889)))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(x_train.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(x_train.shape[2])))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')], shuffle=False)

In [24]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:])
print("predictions shape:", predictions.shape)

predictions = tf.math.argmax(predictions, axis=1)
y_sol = tf.math.argmax(y_test[:], axis=1)
# predictions = [np.argmax(y) for y in predictions[:10]]
# y_sol = [np.argmax(y) for y in y_test[:10]]
print(predictions)
print(y_sol)
m = tf.keras.metrics.SpecificityAtSensitivity(0.5)
m.update_state(predictions, y_sol)
m.result().numpy()

Evaluate on test data
12/12 [==============================] - 0s 12ms/step - loss: 1.3075
test loss, test acc: 1.3075414896011353
Generate predictions for 3 samples
predictions shape: (382, 2)
tf.Tensor(
[1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 0
 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0
 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 0 0 1 1 1 0 0 1 1 0 1 1], shape=(382,), dtyp

0.42708334